<a href="https://colab.research.google.com/github/spyderweb-abdul/Deletion-Detection-in-Unstructured-Data/blob/main/static_missing_link_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
#from google.colab import drive
#drive.mount('/content/drive')
#nb_path = '/content/libraries'
sys.path.append('/content/drive/My Drive/Colab Notebooks/VGRNN/')
sys.path.append('/content/drive/My Drive/Colab Notebooks/')

#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
#sys.path.insert(0,nb_path)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import io
import math
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt 
from scipy.ndimage import rotate
from torch.distributions.uniform import Uniform
from torch.distributions.normal import Normal
#from sklearn.datasets import fetch_mldata
# from torch_geometric import nn as tgnn
from input_data import load_data
from preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, mask_test_edges
import scipy.sparse as sp
from scipy.linalg import block_diag
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import tarfile
import torch.nn.functional as F
import copy
import time

!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

import torch_scatter
from torch_scatter import scatter_mean, scatter_max, scatter_add
from torch_geometric.utils import remove_self_loops, add_self_loops, degree
#from torch_geometric.datasets import Planetoid
import networkx as nx
import scipy.io as sio

import inspect
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, average_precision_score
from sklearn.manifold import TSNE

import copy
import pickle
!pip install sparse
import sparse

import time
import datetime
from datetime import timedelta
import pandas as pd

from torch.utils.tensorboard import SummaryWriter

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

#print(torch.__version__)

In [ ]:
seed = 3
np.random.seed(seed)


# utility functions

def uniform(size, tensor):
    stdv = 1.0 / math.sqrt(size)
    if tensor is not None:
        tensor.data.uniform_(-stdv, stdv)


def glorot(tensor):
    stdv = math.sqrt(6.0 / (tensor.size(0) + tensor.size(1)))
    if tensor is not None:
        tensor.data.uniform_(-stdv, stdv)


def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)


def ones(tensor):
    if tensor is not None:
        tensor.data.fill_(1)


def reset(nn):
    def _reset(item):
        if hasattr(item, 'reset_parameters'):
            item.reset_parameters()

    if nn is not None:
        if hasattr(nn, 'children') and len(list(nn.children())) > 0:
            for item in nn.children():
                _reset(item)
        else:
            _reset(nn)

def tuple_to_array(lot):
    out = np.array(list(lot[0]))
    for i in range(1, len(lot)):
        out = np.vstack((out, np.array(list(lot[i]))))
    
    return out



In [ ]:
# masking functions

def mask_edges_det(adjs_list):
        
    adj = adjs_list

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0
        
    #get the upper trianglar portion of the matrix.
    adj_triu = sp.triu(adj)

    #convert the matrix into a tuple of the format: ((([1, 10]), ([1, 1, 1,..., 1, 1, 1])),...)
    adj_tuple = sparse_to_tuple(adj_triu)

    #get only the 0 index of the tuple. Returns as list: [[1 10],[1 12],[1 4],[20 25]]
    #shape: (n, 2)
    edges = adj_tuple[0]

    #convert the adj sparse matrix to tuple and return the result of the 0 index of the tuple
    edges_all = sparse_to_tuple(adj)[0]

    #get the number of test set: row number(n)/10
    num_test = int(np.floor(edges.shape[0] / 10.))
      
    #get the number of the validation set: row number(n)/20
    num_val = int(np.floor(edges.shape[0] / 20.))
    
    #list numbers of edge index based on the row axis of the edges
    #all_edge_idx = range(edges.shape[0])
    all_edge_idx = list(range(edges.shape[0]))

    #randomize the result
    np.random.shuffle(all_edge_idx)

    #get validation edge index from the randomized edge list. Extract only numbers equal to num_val
    val_edge_idx = all_edge_idx[:num_val]

    #get test edge index from the randomized edge list.
    #Extract only numbers equal to [num_val : (num_val + num_test)]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]

    #get the main test edge set by extracting values fom the edge list indexed by the test_edge_idx list
    test_edges_l = edges[test_edge_idx]
        
    #get the main validation edge set by extracting values fom the edge list indexed by the test_edge_idx list
    val_edges_l = edges[val_edge_idx]

    #delete the stacked test and validation edge set (along the axis=0) from the list of edges. 
    #This will be the training set
    # [[162 165], [162 169], [162 172], [171 174]]

    train_edges_l = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)
        
    #append the list of main edges
    #edges_list.append(edges)
        
    def ismember(a, b, tol=5):
        #Test whether all array elements along a given axis evaluate to True. (np.all)
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)  #np.any evaluate whether any elements evaluate to True

    #get false edge test set
    test_edges_false_l = []
    #Do while test_egde_false list length is still less than the tst_edge list
    while len(test_edges_false_l) < len(test_edges_l):
        #get random integers between 0 (lower) and the row size of the adj (higher)
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])

        #if right and left values are equal, go back to the top loop
        if idx_i == idx_j:
            continue
        #if the tuple of the 2 values are part of edges_all (returns a bool), back to top
        if ismember([idx_i, idx_j], edges_all):
            continue
        #if the empty test_edges_false list is not None, check the conditions
        if test_edges_false_l:
            #if the tuple of the 2 values are part of test_edges_false list, back to top
            if ismember([idx_j, idx_i], np.array(test_edges_false_l)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false_l)):
                continue
        #append result to the test_edges_false list
        test_edges_false_l.append([idx_i, idx_j])  #result sample: [[19, 2], [177, 163], [15, 119], [3, 155],...] 

        
    #get false validation edge set    
    val_edges_false_l = []
    while len(val_edges_false_l) < len(val_edges_l):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges_l):
            continue
        if ismember([idx_j, idx_i], train_edges_l):
            continue
        if ismember([idx_i, idx_j], val_edges_l):
            continue
        if ismember([idx_j, idx_i], val_edges_l):
            continue
        if val_edges_false_l:
            if ismember([idx_j, idx_i], np.array(val_edges_false_l)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false_l)):
                continue
        val_edges_false_l.append([idx_i, idx_j])

    r""" The assert keyword lets you test if a condition in your code returns True, 
    if not, the program will raise an AssertionError.

    #we assert the truthfulness of these conditions. 
    #check to confirm that the values (arg: 1) are bitwise NOT (tilde)
    #in the set of values (arg: 2) in the other list."""

    assert ~ismember(test_edges_false_l, edges_all)
    assert ~ismember(val_edges_false_l, edges_all)
    assert ~ismember(val_edges_l, train_edges_l)
    assert ~ismember(test_edges_l, train_edges_l)
    assert ~ismember(val_edges_l, test_edges_l)

    #get np.ones of elements of the row size of the train_edges
    data = np.ones(train_edges_l.shape[0])

    # Re-build adj matrix for the training set
    r""" [ : , 0 ] means (more or less) [ first_row:last_row , column_0 ]. 
    If you have a 2-dimensional list/matrix/array, this notation will give you all 
    the values in column 0 (from all rows)."""

    adj_train = sp.csr_matrix((data, (train_edges_l[:, 0], train_edges_l[:, 1])), shape=adj.shape)

    #add the new adjacency matrix to its transpose
    adj_train_l = adj_train + adj_train.T


    # NOTE: these edge lists only contain single direction of edge!
    return adj_train_l, train_edges_l, val_edges_l, val_edges_false_l, test_edges_l, test_edges_false_l

def mask_edges_prd(adjs_list):
    pos_edges_l , false_edges_l = [], []
    edges_list = []

    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.        
    adj = adjs_list
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0
        
    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_false = int(edges.shape[0])
        
    pos_edges_l.append(edges)
        
    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)
        
    edges_false = []
    while len(edges_false) < num_false:
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if edges_false:
            if ismember([idx_j, idx_i], np.array(edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(edges_false)):
                continue
        edges_false.append([idx_i, idx_j])

    assert ~ismember(edges_false, edges_all)
        
    false_edges_l.append(edges_false)

    # NOTE: these edge lists only contain single direction of edge!
    return pos_edges_l, false_edges_l


In [ ]:
# loading data

path = 'drive/My Drive/Colab Notebooks/VGRNN/data/'
# # Enron dataset
with open(path+'enron_data_static/enron_static_adj_sparse.pickle', 'rb') as handle:
     adj_sparse_matrix = pickle.load(handle)

with open(path+'enron_data_static/enron_static_adj_dense.pickle', 'rb') as handle:
     adj_dense_matrix = pickle.load(handle)

with open(path+'enron_data_static/enron_static_edge_attribute.pickle', 'rb') as handle:
     edge_attr_matrix = pickle.load(handle)

with open(path+'enron_data_static/enron_static_node_attribute.pickle', 'rb') as handle:
     node_attr_matrix = pickle.load(handle)


outs = mask_edges_det(adj_sparse_matrix)

#reconstructed adjacency matrix of the training set
adj_train_l = outs[0]     

#List of training edge set
train_edges_l = outs[1]         #[1157]        

#List of validation edge set
val_edges_l = outs[2]           #[68]        

#List of false validation edge set(i.e., never exist)
val_edges_false_l = outs[3]     #[68]       

#List of test edge set
test_edges_l = outs[4]          #[136]   

#List of false test edge set 
test_edges_false_l = outs[5]    #[136]  


pos_edges_l, false_edges_l = mask_edges_prd(adj_sparse_matrix)

# creating edge list
edge_idx_list = torch.tensor(np.transpose(train_edges_l), dtype=torch.long)

#print(edge_idx_list)

In [ ]:
# layers

class E_GCN_Conv(nn.Module):
    def __init__(self, in_channels, out_channels, act=F.relu, improved=True, bias=True, num_channels=10, aggr='sum'):
        super(E_GCN_Conv, self).__init__()

        self.in_channels = in_channels            #[64]
        self.out_channels = out_channels          #[32]
        self.act = act
        self.num_channels = num_channels

        self.weight = Parameter(torch.Tensor(in_channels, out_channels, num_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels, num_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

        if (aggr == 'concat'):
            self.aggr = 'concat'
            self.last_ops = nn.Linear(self.out_channels * self.num_channels, self.out_channels)
        elif (aggr == 'sum'):
            self.aggr = 'sum'
            self.last_ops = nn.Linear(self.out_channels, self.out_channels)
        

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)

    
    def forward(self, x, edge_index, edge_attr):

        #add or remove node self loop. We remove in our case
        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)

        
        #edge index rows and column representation
        row, col = edge_index     #[21]

        #normalize the adjacency matrix
        #deg = scatter_add(edge_attr, row, dim=0, dim_size=x.size(0))
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        #reshape the row and column vectors
        deg_inv_sqrt_row = deg_inv_sqrt[row].view(-1, 1)      #[[1.0000],[1.0000]]
        deg_inv_sqrt_col = deg_inv_sqrt[col].view(-1, 1)      #[[0.5774],[0.0000]]

        #multiply row and col vectors with edge weights (We replace the adjacency matrix with the edge tensor)
        norm_edge = deg_inv_sqrt_row * edge_attr * deg_inv_sqrt_col     #size([edge_index[row/col] No., 14])


        #Slice and list the normalized vectors based on the nu. of channels
        norm = []
        for i in range(0, edge_attr.size()[1]):
            norm.append(norm_edge[:, i:i+1])

        node_state_list = []
        #for each edge channels, we perform a weoghted convolution with edge weights as co-efficient
        for c in range(self.num_channels):
            if self.in_channels > self.out_channels:

                #if the weight matrix is not none
                if self.weight is not None:
                    #matrix product of the node (hidden state) with the weight matrix
                    weighted_nodes = torch.matmul(x, self.weight[:, :, c])      #(size[149, 32])
                else:
                    #otherwise, hidden state remains same
                    weighted_nodes = x
                
                #if vectors are normalized
                if norm is not None: 
                    #multiply each element in the each channels of the norm with weighted hidden state             
                    weighted_conv = torch.mul(norm[c], weighted_nodes[row])      #size(21, 32)

                    #propagate messages through all edges and update the nodes
                    weighted_conv_sum = scatter_add(weighted_conv, col, dim=0, dim_size=x.size(0)) #size(149, 32)
                else:
                    weighted_conv_sum = scatter_add(weighted_nodes[row], col, dim=0, dim_size=x.size(0))

                channel_node_state = weighted_conv_sum

            else:
                if norm is not None:
                    unweighted_conv = torch.mul(norm[c], x[row])
                    unweighted_conv_sum = scatter_add(unweighted_conv, col, dim=0, dim_size=x.size(0))
                else:
                    unweighted_conv_sum = scatter_add(x[row], col, dim=0, dim_size=x.size(0))
                
                if self.weight is not None:
                    channel_node_state = torch.matmul(unweighted_conv_sum.float(), self.weight[:, :, c])
            
            #add linear bias if True
            if self.bias is not None:
                channel_node_state = channel_node_state + self.bias[:, c]
            
            #pass param through a linear activation function
            channel_node_state = self.act(channel_node_state)
            #append each channel to node state list
            node_state_list.append(channel_node_state)        #size(N, 32/16)

        #we consider two aggregation method across each channels of the edge weights
        #1. Sum aggregation method 
        if (self.aggr == 'sum'):
            node_states = torch.stack(node_state_list, dim=1).sum(1).float()     #[N, 32]
        #2. Concat aggregation method               
        elif (self.aggr == 'concat'): 
            node_states = torch.cat(node_state_list, dim=1).float()
            
        #pass aggregated vectors through a flexible linear transformation layer      
        out = self.last_ops(node_states)                        #size(N, 32/16)

        return out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels, 
                                   self.out_channels, self.num_channels)

In [ ]:
class gru_gcn(nn.Module):
    def __init__(self, input_size, hidden_size, n_layer, bias=True):
        super(gru_gcn, self).__init__()

        self.hidden_size = hidden_size
        self.n_layer = n_layer
        
        # gru weights
        self.weight_xz = []
        self.weight_hz = []
        self.weight_xr = []
        self.weight_hr = []
        self.weight_xh = []
        self.weight_hh = []
        
        for i in range(self.n_layer):
            if i==0:
                self.weight_xz.append(E_GCN_Conv(input_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_hz.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_xr.append(E_GCN_Conv(input_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_hr.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_xh.append(E_GCN_Conv(input_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_hh.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
            else:
                self.weight_xz.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_hz.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_xr.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_hr.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_xh.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
                self.weight_hh.append(E_GCN_Conv(hidden_size, hidden_size, act=lambda x:x, bias=bias))
    
    def forward(self, inp, edge_index, edge_tensor, h):
        h_out = torch.zeros(h.size())
        for i in range(self.n_layer):
            if i==0:
                
                z_g = torch.sigmoid(self.weight_xz[i](inp, edge_index, edge_tensor) + self.weight_hz[i](h[i], edge_index, edge_tensor))
                r_g = torch.sigmoid(self.weight_xr[i](inp, edge_index, edge_tensor) + self.weight_hr[i](h[i], edge_index, edge_tensor))
                h_tilde_g = torch.tanh(self.weight_xh[i](inp, edge_index, edge_tensor) + self.weight_hh[i](r_g * h[i], edge_index, edge_tensor))
                h_out[i] = z_g * h[i][0: inp.size(0)] + (1 - z_g) * h_tilde_g
        #         out = self.decoder(h_t.view(1,-1))
            else:
                z_g = torch.sigmoid(self.weight_xz[i](h_out[i-1], edge_index, edge_tensor) + self.weight_hz[i](h[i], edge_index, edge_tensor))
                r_g = torch.sigmoid(self.weight_xr[i](h_out[i-1], edge_index, edge_tensor) + self.weight_hr[i](h[i], edge_index, edge_tensor))
                h_tilde_g = torch.tanh(self.weight_xh[i](h_out[i-1], edge_index, edge_tensor) + self.weight_hh[i](r_g * h[i], edge_index, edge_tensor))
                h_out[i] = z_g * h[i] + (1 - z_g) * h_tilde_g
        #         out = self.decoder(h_t.view(1,-1))
        
        out = h_out
        return out, h_out


In [ ]:
# VGRNN model

class VGAE_Edge(nn.Module):
    def __init__(self, node_feat_dim, hidden_dim, latent_var_dim, n_layers, eps, conv='GCN', bias=False):
        super(VGAE_Edge, self).__init__()
        
        #input dimension
        self.node_feat_dim = node_feat_dim        
        self.eps = eps
        #hidden_layer dim.
        self.hidden_dim = hidden_dim        #32
        #latent variable dim.
        self.latent_var_dim = latent_var_dim        #10
        self.n_layers = n_layers   #1
       
        if conv == 'GCN':
            #flexible sequential neural network linear transformations
            self.input_emb = nn.Sequential(nn.Linear(node_feat_dim, hidden_dim), nn.ReLU())
            self.output_emb = nn.Sequential(nn.Linear(latent_var_dim, hidden_dim), nn.ReLU())
            
            #encoder functions
            self.encoder = E_GCN_Conv(hidden_dim + hidden_dim, hidden_dim)            
            self.encoder_mu = E_GCN_Conv(hidden_dim, latent_var_dim, act=lambda x:x)
            self.encoder_sigma = E_GCN_Conv(hidden_dim, latent_var_dim, act=F.softplus)
          

    def forward(self, x, edge_idx_list, edge_attr_matrix, adj_dense_matrix, hidden_in=None):

        #assert the length of edge matrix = elngth of the edge indices
        #assert len(edge_attr_matrix) == len(edge_idx_list)

        #initialize params
        kld_loss = 0
        nll_loss = 0

        #hidden var will be none in the first set of operations
        if hidden_in is None:
            #so we create a matrix of zeros as initial representation
            h = Variable(torch.zeros(self.n_layers, x.size(0), self.hidden_dim))  #size([1, 149, 32])
        else:
            #hidden var here will be the recurrent vectors
            h = Variable(hidden_in)

        #linearly transform x features
        input_emb = self.input_emb(x.float())              #[149, 32]
        
        edge_tensor = (edge_attr_matrix[edge_idx_list[0], edge_idx_list[1]])
           
        #encoder
        encod = self.encoder(torch.cat([input_emb, h[-1]], 1), edge_idx_list, edge_tensor)    #[149, 32]
        #encoder mean
        encod_mu = self.encoder_mu(encod, edge_idx_list, edge_tensor)                   #[149, 10]
        #encoder standar deviation
        encod_sigma = self.encoder_sigma(encod, edge_idx_list, edge_tensor)                     #[149, 14]
            
            
        #sampling and reparameterization
        z = self._reparameterized_sample(encod_mu, encod_sigma)  #[149, 10]
        #apply a fully connected layer to z_t
        output_emb = self.output_emb(z)                                  #[149, 32]

        #decoder function -> takes the linearly transformed latent variable and egde indices as args
        decoded = self.dec(output_emb, edge_idx_list)
            
        num_nodes = adj_dense_matrix.size(0)
        encoder_mu_slice = encod_mu[0:num_nodes, :]
        encoder_sigma_slice = encod_sigma[0:num_nodes, :]
        adj_decoder = decoded[0:num_nodes, 0:num_nodes]

        #computing losses
        kld_loss += self.kl_divergence(encoder_mu_slice, encoder_sigma_slice)
        nll_loss += self.nll_bernoulli(adj_decoder, adj_dense_matrix)

        return kld_loss, nll_loss, adj_decoder, h
    
    #decoder function
    def dec(self, z, edge_index):
        outputs = Decoder(act=lambda x:x)(z, edge_index)
        return outputs
    
    def reset_parameters(self, stdv=1e-1):
        for weight in self.parameters():
            weight.data.normal_(0, stdv)
     
    def _init_weights(self, stdv):
        pass
    
    def _reparameterized_sample(self, mean, std):
        eps1 = torch.FloatTensor(std.size()).normal_()
        eps1 = Variable(eps1)
        return eps1.mul(std).add_(mean)
        
    def kl_divergence(self, mu, sigma):        
        mu_size = mu.size(0)
        sigma_log = torch.log(sigma + self.eps)
        kld_element =  (1 + 2*sigma_log - (sigma**2) - (mu**2))
        kld_element = kld_element.detach().numpy()      
        kld_element = torch.tensor(np.nan_to_num(kld_element, copy=True, nan=0.0))        
        return (-0.5 / mu_size) * kld_element.sum(1).mean()
    
    #negative log likelihood bernoulli
    def nll_bernoulli(self, logits, target):                     
        positive_weight = float(target.size(0) * target.size(0) - target.sum()) / target.sum()   #negative samples/positive samples
        #norm_weight = target.size(0) * target.size(0) / float((target.size(0) * target.size(0) - target.sum())*2) 
        bce = F.binary_cross_entropy_with_logits(logits, target, pos_weight=positive_weight, reduction='mean')
        
        nll_loss = (-1.0 / target.size(0)) * bce
        return - nll_loss

In [ ]:
class Decoder(nn.Module):
    def __init__(self, act=torch.sigmoid):
        super(Decoder, self).__init__()        

        self.act = act
    def forward(self, z, edge_index):
        z = F.dropout(z, p=0., training=True) 

        a_hat = torch.transpose(z, dim0=0, dim1=1)
        a_hat = self.act(torch.mm(z, a_hat))

        a_hat_z0 = z[edge_index[0]]
        a_hat_z1 = z[edge_index[1]]

        link_prob = (a_hat_z0 * a_hat_z1).sum(dim=1)
        adj_prob = self.act(link_prob) if self.act else link_prob

        return a_hat



In [ ]:
r""" Calculate and evaluate the Area Under (Receiver Operating Characteristic) Curve 
     and the Average Precision (AP) """

# evaluation function
def get_eval_scores(edges_pos, edges_neg, adj_dense_matrix, a_embs):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    #print(edges_pos[0])

    #print(edges_neg[0])

    #(from gradient descent) with detach and change back to numpy
    a_emb = a_embs.detach().numpy()

    #reconstruct the adjacency matrix of the embeddings
    adj_emb = np.dot(a_emb, a_emb.T)          #[149, 149]

    adj_dense = adj_dense_matrix
       
    #initialize predicted edge list
    pos_adj, pred_pos_adj = [], []

    for e in edges_pos[0]:
        pred_pos_adj.append(sigmoid(adj_emb[e[0], e[1]]))
        pos_adj.append(adj_dense[e[0], e[1]])         

    #print('Pos: ', pos_adj)
    #print('\nPred: ', pred_pos_adj)

    neg_adj, pred_neg_adj = [], []

    for e in edges_neg[0]:
        pred_neg_adj.append(sigmoid(adj_emb[e[0], e[1]]))
        neg_adj.append(adj_dense[e[0], e[1]])

    #stack up the positive and negative predicted features
    all_pred_adj = np.hstack([pred_pos_adj, pred_neg_adj])       
    all_true_adj = np.hstack([np.ones(len(pred_pos_adj)), np.zeros(len(pred_neg_adj))])

    auc = roc_auc_score(all_true_adj, all_pred_adj)
    ap = average_precision_score(all_true_adj, all_pred_adj)

    return auc, ap, neg_adj, pred_neg_adj

In [ ]:
# hyperparameters

hidden_dim = 32
latent_var_dim = 16
n_layers =  1
clip = 10
learning_rate = 1e-2
node_feat_dim = node_attr_matrix.shape[1]
timesteps_len = len(train_edges_l)  #1157
eps = 1e-10
conv_type='GCN'


node_attr = torch.tensor(node_attr_matrix)    #[149, 6]

adj_label_l = torch.tensor(adj_train_l.toarray().astype(np.float32))


# building model
model = VGAE_Edge(node_feat_dim, hidden_dim, latent_var_dim, n_layers, eps, conv=conv_type, bias=True)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# training
test_init = 0

#writer = SummaryWriter('drive/MyDrive/Colab Notebooks/VGRNN/tensorboard_log/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

start_time = time.monotonic()
for k in range(50):
    optimizer.zero_grad()

    kld_loss, nll_loss, _, hidden = model(node_attr, edge_idx_list, edge_attr_matrix, adj_dense_matrix)    

    loss = kld_loss + nll_loss
    loss.backward()
    optimizer.step()
    
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    
    if k>test_init:
        _, _, adj_dec, _ = model(node_attr, edge_idx_list, edge_attr_matrix, adj_label_l, hidden)                             

        auc, ap, pos, pred  = get_eval_scores(pos_edges_l, false_edges_l, adj_dense_matrix, adj_dec)

        #writer.add_scalar("Loss/train", loss.mean().item(), k)

    #Note: Prior mean reduces the loss than the decoded variables. 
    print('********************************************************')
    print('epoch: ', k)
    print('\nLOSS => kld_loss: {} | nll_loss: {} | loss: {}'.format(round(kld_loss.mean().item(), 4)
                                                                    , round(nll_loss.mean().item(), 4)
                                                                    , round(loss.mean().item(), 4)
                                                                    ))
    
    if k>test_init:
        #writer.add_scalar("mean_score", np.mean(np.array(square_mean_score)), k)
        #writer.add_scalar("root_mean_score", np.mean(np.array(root_mean_score)), k)
        #writer.add_scalar("mean_abs_score", np.mean(np.array(mean_absolute_score)), k)
        #writer.add_scalar("mape", np.mean(np.array(mape)), k)
        #writer.add_scalar("r2_score", np.mean(np.array(r2)), k)

        print('\nADJ. RECONSTRUCTION => auc_mean: {} | ap_mean: {}'.format(round(np.mean(np.array(auc)), 4)
                                                                          , round(np.mean(np.array(ap)), 4)
                                                                          ))
#writer.flush() 
#writer.close()   
end_time = time.monotonic()
print('Total Execution Time: {}'.format(timedelta(seconds=end_time - start_time)))

#!pip install tensorboard

#!tensorboard --logdir='drive/MyDrive/Colab Notebooks/VGRNN/tensorboard_log/'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


********************************************************
epoch:  0

LOSS => kld_loss: 0.9 | nll_loss: 0.4389 | loss: 1.3389
********************************************************
epoch:  1

LOSS => kld_loss: 0.7813 | nll_loss: 0.7757 | loss: 1.557

ADJ. RECONSTRUCTION => auc_mean: 0.5 | ap_mean: 0.5
********************************************************
epoch:  2

LOSS => kld_loss: 0.2814 | nll_loss: 0.1602 | loss: 0.4416

ADJ. RECONSTRUCTION => auc_mean: 0.5 | ap_mean: 0.5
********************************************************
epoch:  3

LOSS => kld_loss: 0.2142 | nll_loss: 0.1265 | loss: 0.3407

ADJ. RECONSTRUCTION => auc_mean: 0.5 | ap_mean: 0.5
********************************************************
epoch:  4

LOSS => kld_loss: 0.1376 | nll_loss: 0.0472 | loss: 0.1848

ADJ. RECONSTRUCTION => auc_mean: 0.5 | ap_mean: 0.5
********************************************************
epoch:  5

LOSS => kld_loss: 0.0922 | nll_loss: 0.0269 | loss: 0.1191

ADJ. RECONSTRUCTION => auc_me